In [ ]:
import csv
import cv2
import sklearn
import numpy as np
from keras_tqdm import TQDMNotebookCallback
from log_progress import log_progress

import sys
sys.stdout.isatty()


In [ ]:
lines = []

paths_to_import = ['D:\\Car Engine\\Attempt on Track 2\\']

samples = []

correction = 0.2

flipped = True
unflipped = False
for base_path in paths_to_import:
    with open(base_path + 'driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        i = 0
        for line in reader:
            center_path = line[0]
            left_path = line[1]
            right_path = line[2]
            steering_center = float(line[3])
            samples.append((base_path,center_path, steering_center, unflipped))
            samples.append((base_path,left_path, steering_center + correction, unflipped))
            samples.append((base_path,right_path, steering_center - correction, unflipped))

            #samples.append((center_path, steering_center * -1, flipped))
            #samples.append((left_path, (steering_center + correction) * -1, flipped))
            #samples.append((right_path, (steering_center - correction) * -1, flipped))

            i+=1
            if i % 1000 == 0:
                print(i)


In [ ]:
def process_image(x):
    return x/127.5 - 1.

def load_image(sample):
    name = sample[0] + 'IMG\\'+ sample[1].split("\\")[-1]
    img = cv2.imread( name)
    if img is None:
        print("Name is bad", name)
        awd
    return img
#    return process_image(cv2.imread(name))

X_train = np.array([load_image(sample) for sample in log_progress(samples)])
Y_train = np.array([sample[2] for sample in log_progress(samples)])

In [ ]:
X_train[0].shape

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

img = X_train[0].copy()

start = 55
end = 130
height = img.shape[1]
width = img.shape[0]



img[start:end] = img[start:end] * 0.5
plt.figure()
plt.imshow(img)

img = X_train[0][start:end]
print(img.shape)
plt.figure()
plt.imshow(img)

In [ ]:
ch, row, col = 3, 160, 320

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Cropping2D, Conv2D
model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(row, col, ch)))
model.add(Cropping2D(cropping=((40,15),(0,0))))

model.add(Conv2D(24,5,5, subsample=(2,2), activation="relu"))
model.add(Conv2D(36,5,5, subsample=(2,2), activation="relu"))
model.add(Conv2D(48,5,5, subsample=(2,2), activation="relu"))

model.add(Conv2D(48,3,3, activation="relu"))
model.add(Conv2D(48,3,3, activation="relu"))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

In [ ]:
if True:
    print("Fitting, non terminal")
    model.fit(X_train[0:500], Y_train[0:500], epochs=3, shuffle=True)
else:
    model.fit(X_train[0:500], Y_train[0:500], validation_split=0.2, verbose=0, callbacks=[TQDMNotebookCallback()], epochs=5, shuffle=True)


In [ ]:
print("Saving")
model_name = './model_2.h5'
model.save(model_name)
print("Saved: ", model_name)